# Python Bootcamp

Add dataset






In [ ]:
!gdown --id 1x9Y3xylEDX5aS3EXa6qrS0gEscFzWafQ
!gdown --id 1OnIYNtdVIVIgrxdDkL3L0chpG8DpWTjz
# !gdown --id 14iCgWhaiPEzUBEMg9-CV_qxAOTScKGwI

Downloading...
From: https://drive.google.com/uc?id=1x9Y3xylEDX5aS3EXa6qrS0gEscFzWafQ
To: /content/patients.dat
100% 5.98k/5.98k [00:00<00:00, 5.12MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OnIYNtdVIVIgrxdDkL3L0chpG8DpWTjz
To: /content/fram.txt
100% 1.46M/1.46M [00:00<00:00, 92.6MB/s]


In [ ]:
!pip install scipy
!pip install statsmodels
!pip install xlsxwriter
!pip install xlrd

     |████████████████████████████████| 153kB 6.5MB/s 


In [ ]:
import numpy as np
from pprint import pprint
from prettytable import PrettyTable
import pandas as pd
import scipy
from scipy import stats
from scipy import sparse
import re
import xlsxwriter


## Recap 
### Numeric Arrays
- e.g. list of phone numbers
- B = [4670456, 7640600, 123456];

### Matrices
- e.g. list of GPS coordinates
- A  = [42.3086,-83.6921; 82.208, 23.692];

### Cell Arrays
- e.g. list of names
- C = {'Duck','Goose','Crane'};

## Matrix Manipulation
### Adding a scalar to an array

In [ ]:
A = np.array([[1],
              [2],
              [3],
              [4]])
A + 3

array([[4],
       [5],
       [6],
       [7]])

### Adding two arrays

In [ ]:
B = [[3],
     [4],
     [5],
     [6]]
A + B

array([[ 4],
       [ 6],
       [ 8],
       [10]])

### Element-wise multiplication

In [ ]:
A*B

array([[ 3],
       [ 8],
       [15],
       [24]])

Matrix multiplication

In [ ]:
C = [[5,10,15,20]]
A@ C

array([[ 5, 10, 15, 20],
       [10, 20, 30, 40],
       [15, 30, 45, 60],
       [20, 40, 60, 80]])

## Organizing variables into structures
### Strucutures 
Structure arrays can be used to group related data together. The data in structure arrays is accessed by name.

In [ ]:
class Patient():
  pass

patient = Patient()
patient.name = "Jane Smith"
patient.age = 28
patient.results = np.array([[68, 70, 68], [72, 81, 69], [172, 170, 169]])
vars(patient)

{'age': 28, 'name': 'Jane Smith', 'results': array([[ 68,  70,  68],
        [ 72,  81,  69],
        [172, 170, 169]])}

## Tables
Tables are used for storing data in rows and column-oriented variables.
- Tables can contain different data types, such as strings and doubles
- Each variable in the table must have the same number of rows

<!-- Use `readtable` to import example patient data and then `summary` to examine its contents: -->

In [ ]:
with open('patients.dat') as f:
    rows = [line.split(",") for line in f]  

In [ ]:
T = pd.DataFrame(np.array(rows[1:]), columns=rows[0])
T[:4]

,LastName,Gender,Age,Location,Height,Weight,Smoker,Systolic,Diastolic,SelfAssessedHealthStatus\n
0,Smith,Male,38,County General Hospital,71,176,1,124,93,Excellent\n
1,Johnson,Male,43,VA Hospital,69,163,0,109,77,Fair\n
2,Williams,Female,38,St. Mary's Medical Center,64,131,0,125,83,Good\n
3,Jones,Female,40,VA Hospital,67,133,0,117,75,Fair\n


Now create a table that only includes the patieent age, height and weight:

In [ ]:
T2 = T[["Age", "Height", "Weight"]]

## Framington Heart Disease Dataset
This dataset comes from a landmark study that analyzed ~14,000 people from three generations. The findings have informed the understanding of factors that impact cardiovascular health.  

Import the dataset and determine its size:

In [ ]:
fram = pd.read_csv("fram.txt", delimiter=",")
fram.shape

(11627, 39)

The output of size indiciates that the dataset includes 11,627 rows and 39 columns. Display the names of the 39 columns:


In [ ]:
fram.columns

Index(['RANDID', 'SEX', 'TOTCHOL', 'AGE', 'SYSBP', 'DIABP', 'CURSMOKE',
       'CIGPDAY', 'BMI', 'DIABETES', 'BPMEDS', 'HEARTRTE', 'GLUCOSE', 'educ',
       'PREVCHD', 'PREVAP', 'PREVMI', 'PREVSTRK', 'PREVHYP', 'TIME', 'PERIOD',
       'HDLC', 'LDLC', 'DEATH', 'ANGINA', 'HOSPMI', 'MI_FCHD', 'ANYCHD',
       'STROKE', 'CVD', 'HYPERTEN', 'TIMEAP', 'TIMEMI', 'TIMEMIFC', 'TIMECHD',
       'TIMESTRK', 'TIMECVD', 'TIMEDTH', 'TIMEHYP'],
      dtype='object')

## Extracting Simple Properties
Determine the mean, median and range of BMI:

In [ ]:
fram["BMI"].mean(skipna=False)

nan

In [ ]:
fram["BMI"].median(skipna=False)

nan

The mean and median functions return NaN because there is data missing from the BMI array. MATLAB also has functions that ignore these missing data points:


In [ ]:
fram["BMI"].mean(skipna=True)

25.87734859611223

In [ ]:
fram["BMI"].median(skipna=True)

25.48

We will now look at extracting data from the Framingham data set based on certain criteria. For example, what if we only want to examine individuals who smoke?

In [ ]:
fram[fram["CURSMOKE"]==1]

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,educ,PREVCHD,PREVAP,PREVMI,PREVSTRK,PREVHYP,TIME,PERIOD,HDLC,LDLC,DEATH,ANGINA,HOSPMI,MI_FCHD,ANYCHD,STROKE,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
5,9428,1,245.0,48,127.5,80.0,1,20.0,25.34,0,0.0,75.0,70.0,1.0,0,0,0,0,0,0,1,NaN,NaN,0,0,0,0,0,0,0,0,8766,8766,8766,8766,8766,8766,8766,8766
6,9428,1,283.0,54,141.0,89.0,1,30.0,25.34,0,0.0,75.0,87.0,1.0,0,0,0,0,0,2199,2,NaN,NaN,0,0,0,0,0,0,0,0,8766,8766,8766,8766,8766,8766,8766,8766
7,10552,2,225.0,61,150.0,95.0,1,30.0,28.58,0,0.0,65.0,103.0,3.0,0,0,0,0,1,0,1,NaN,NaN,1,0,0,0,0,1,1,1,2956,2956,2956,2956,2089,2089,2956,0
8,10552,2,232.0,67,183.0,109.0,1,20.0,30.18,0,0.0,60.0,89.0,3.0,0,0,0,0,1,1977,2,NaN,NaN,1,0,0,0,0,1,1,1,2956,2956,2956,2956,2089,2089,2956,0
9,11252,2,285.0,46,130.0,84.0,1,23.0,23.10,0,0.0,85.0,85.0,3.0,0,0,0,0,0,0,1,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,4285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11617,9993179,2,NaN,50,131.0,80.0,1,25.0,21.22,0,0.0,75.0,NaN,1.0,0,0,0,0,0,2226,2,NaN,NaN,1,0,0,0,0,0,0,1,6729,6729,6729,6729,6729,6729,6729,4396
11618,9993179,2,251.0,56,145.0,92.0,1,35.0,21.97,0,1.0,95.0,90.0,1.0,0,0,0,0,1,4396,3,70.0,181.0,1,0,0,0,0,0,0,1,6729,6729,6729,6729,6729,6729,6729,4396
11624,9999312,2,196.0,39,133.0,86.0,1,30.0,20.91,0,0.0,85.0,80.0,3.0,0,0,0,0,0,0,1,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,4201
11625,9999312,2,240.0,46,138.0,79.0,1,20.0,26.39,0,0.0,90.0,83.0,3.0,0,0,0,0,0,2390,2,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,4201


Similarly, we can pick out individuals with a BMI above 35:


In [ ]:
fram[fram["BMI"]>35]

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,educ,PREVCHD,PREVAP,PREVMI,PREVSTRK,PREVHYP,TIME,PERIOD,HDLC,LDLC,DEATH,ANGINA,HOSPMI,MI_FCHD,ANYCHD,STROKE,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
16,12629,2,220.0,70,149.0,81.0,0,0.0,36.76,0,0.0,80.0,98.0,1.0,1,1,0,0,1,2212,2,NaN,NaN,0,1,0,0,1,0,0,1,373,8766,8766,373,8766,8766,8766,2212
54,43522,2,NaN,55,129.0,76.0,0,0.0,35.71,0,0.0,76.0,NaN,1.0,0,0,0,0,0,4403,3,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,5226
92,82188,1,225.0,37,124.5,92.5,0,0.0,38.53,0,0.0,95.0,83.0,2.0,0,0,0,0,1,0,1,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,0
93,82188,1,244.0,43,156.0,109.0,0,0.0,38.53,0,0.0,85.0,67.0,2.0,0,0,0,0,1,2163,2,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,0
94,82188,1,226.0,49,190.0,123.0,0,0.0,45.43,0,1.0,100.0,73.0,2.0,0,0,0,0,1,4303,3,34.0,192.0,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11498,9887608,2,240.0,47,139.0,80.0,0,0.0,38.44,0,1.0,70.0,NaN,1.0,0,0,0,0,1,2152,2,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,0
11563,9941222,2,236.0,63,155.0,82.0,0,0.0,39.17,1,0.0,78.0,79.0,1.0,0,0,0,0,1,0,1,NaN,NaN,1,0,1,1,1,0,1,1,5538,5380,5380,5380,5538,5380,5538,0
11564,9941222,2,276.0,69,178.0,72.0,0,0.0,39.64,1,0.0,80.0,156.0,1.0,0,0,0,0,1,2153,2,NaN,NaN,1,0,1,1,1,0,1,1,5538,5380,5380,5380,5538,5380,5538,0
11595,9969773,2,260.0,50,190.0,130.0,0,0.0,43.67,1,0.0,85.0,260.0,1.0,0,0,0,0,1,0,1,NaN,NaN,0,1,1,1,1,0,1,1,8427,7994,7994,7994,8766,7994,8766,0


This selection criteria method can also be applied to multiple variables at once:


In [ ]:
SMOKER_BMI_rows = (fram["CURSMOKE"]==1) & (fram["BMI"]>35)
fram[SMOKER_BMI_rows]


,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,educ,PREVCHD,PREVAP,PREVMI,PREVSTRK,PREVHYP,TIME,PERIOD,HDLC,LDLC,DEATH,ANGINA,HOSPMI,MI_FCHD,ANYCHD,STROKE,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
192,174973,2,206.0,42,130.0,80.0,1,3.0,37.51,0,0.0,70.0,83.0,NaN,0,0,0,0,0,1865,2,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,8021
193,174973,2,208.0,48,122.0,74.0,1,3.0,40.33,0,0.0,75.0,82.0,NaN,0,0,0,0,0,4028,3,35.0,173.0,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,8021
231,202101,2,326.0,61,200.0,104.0,1,1.0,38.46,0,0.0,57.0,78.0,2.0,0,0,0,0,1,0,1,NaN,NaN,1,0,0,0,0,0,0,1,7493,7493,7493,7493,7493,7493,7493,0
672,610021,2,180.0,60,200.0,122.5,1,20.0,44.27,1,0.0,88.0,150.0,1.0,0,0,0,0,0,0,1,NaN,NaN,1,0,0,0,0,1,1,1,6735,6735,6735,6735,5475,5475,6735,1491
1102,935116,1,229.0,44,177.5,120.0,1,10.0,39.88,0,0.0,104.0,78.0,1.0,0,0,0,0,1,0,1,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10245,8875547,2,254.0,57,146.5,81.0,1,1.0,41.61,0,0.0,72.0,85.0,1.0,0,0,0,0,1,0,1,NaN,NaN,1,1,0,1,1,0,1,1,3024,4494,1936,1936,4494,1936,4494,0
10246,8875547,2,199.0,64,163.5,90.0,1,4.0,35.37,1,0.0,95.0,130.0,1.0,1,0,1,0,1,2390,2,NaN,NaN,1,1,0,1,1,0,1,1,3024,4494,1936,1936,4494,1936,4494,0
10531,9079405,2,358.0,62,215.0,110.0,1,20.0,37.62,1,1.0,110.0,368.0,3.0,0,0,0,0,1,0,1,NaN,NaN,1,0,0,0,0,1,1,1,53,53,53,53,47,47,53,0
10941,9392174,2,246.0,54,153.0,80.0,1,10.0,37.30,0,0.0,92.0,74.0,2.0,0,0,0,0,1,0,1,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,0


By scrolling through the CURSMOKE and BMI columns of the above table, we can confirm that the new table only includes data for individuals that both smoked and have a BMI above 35. Now create a table which only includes the age, cholesterol and heartrate for these individuals:

In [ ]:
new_table = fram.loc[SMOKER_BMI_rows,['AGE','TOTCHOL','HEARTRTE']]
new_table

,AGE,TOTCHOL,HEARTRTE
192,42,206.0,70.0
193,48,208.0,75.0
231,61,326.0,57.0
672,60,180.0,88.0
1102,44,229.0,104.0
...,...,...,...
10245,57,254.0,72.0
10246,64,199.0,95.0
10531,62,358.0,110.0
10941,54,246.0,92.0


We can also convert this table to a matrix, but we will lose the table headers, as MATLAB arrays can only contain one type of data

In [ ]:
new_array = new_table.to_numpy()
print(new_array.shape)
print(new_array)

(84, 3)
[[ 42. 206.  70.]
 [ 48. 208.  75.]
 [ 61. 326.  57.]
 [ 60. 180.  88.]
 [ 44. 229. 104.]
 [ 56. 153.  75.]
 [ 48. 268.  72.]
 [ 60. 247.  78.]
 [ 73.  nan  nan]
 [ 55. 233.  80.]
 [ 67. 192.  55.]
 [ 42. 199.  80.]
 [ 46. 212. 110.]
 [ 51. 250.  94.]
 [ 62. 206.  80.]
 [ 36. 248.  65.]
 [ 42. 303.  86.]
 [ 48. 263.  96.]
 [ 54. 241.  69.]
 [ 39. 310.  69.]
 [ 69. 335.  80.]
 [ 40. 229.  66.]
 [ 43. 174.  82.]
 [ 49. 180.  80.]
 [ 41. 306. 100.]
 [ 47. 270.  97.]
 [ 53.  nan 118.]
 [ 61. 218.  90.]
 [ 47. 221.  85.]
 [ 57. 289.  68.]
 [ 59. 201.  65.]
 [ 50.  nan  72.]
 [ 42. 240.  75.]
 [ 55. 280.  90.]
 [ 54. 265.  96.]
 [ 61. 312.  85.]
 [ 65. 211.  80.]
 [ 71. 220.  78.]
 [ 37. 274.  68.]
 [ 43. 374.  78.]
 [ 49. 209.  82.]
 [ 41. 280. 110.]
 [ 47. 270.  80.]
 [ 50. 223.  85.]
 [ 59. 312. 120.]
 [ 67. 270.  85.]
 [ 53. 251.  85.]
 [ 49. 288.  73.]
 [ 55. 252.  85.]
 [ 41. 240.  90.]
 [ 46. 286.  75.]
 [ 50. 213.  95.]
 [ 56. 185.  92.]
 [ 48. 172.  75.]
 [ 57. 283. 100.]
 [


## Manipulating Datasets
As seen earlier in the BMI array, there are missing values throughout the dataset, which are signified by NaN. It is important to know how to both find and replace these missing values.
We can use the find and isnan functions to determine the rows where data is missing:

In [ ]:
fram.index[fram["BMI"].isnull()]

Int64Index([    1,   264,   346,   432,   794,  1215,  1299,  1526,  1893,
             2630,  3123,  3138,  3500,  3712,  3806,  4340,  4341,  4342,
             4369,  4420,  4421,  4757,  4758,  5398,  5601,  5651,  5652,
             5653,  5718,  5917,  5944,  6024,  6149,  6597,  6828,  6912,
             7322,  7335,  7435,  7998,  7999,  8000,  8440,  8455,  8861,
             9142,  9797, 10232, 10250, 10694, 11347, 11367],
           dtype='int64')

Now we want to create a new array for BMI without the missing data points:

In [ ]:
new_BMI = fram["BMI"];
new_BMI = new_BMI[~fram["BMI"].isnull()]

To confirm that this worked, we can check that the size of the BMI array has decreased (see Workspace). Also, the mean function now works correctly:

In [ ]:
new_BMI.mean()

25.87734859611223

Now that the missing data points have been removed, sort the BMI array:


In [ ]:
new_BMI.sort_values()


7875     14.43
6320     14.53
1454     15.16
4114     15.32
8451     15.33
         ...  
208      52.94
10762    55.31
7272     56.80
7270     56.80
7271     56.80
Name: BMI, Length: 11575, dtype: float64

Next, let's look at the outliers in the BMI array:


In [ ]:
# This is a z-score outlier algorithm
z = np.abs(stats.zscore(new_BMI))
threshold = 3
outliers = np.where(z > threshold)
outliers

(array([   91,    92,    93,    97,   192,   206,   207,   208,   230,
          292,   308,   309,   310,   382,   383,   435,   436,   668,
          669,  1097,  1099,  1161,  1162,  1163,  1292,  1477,  1555,
         1803,  1805,  2008,  2009,  2222,  2223,  2233,  2390,  2488,
         2596,  2597,  2598,  3123,  3189,  3480,  4118,  4119,  4128,
         4129,  4130,  4320,  4321,  4575,  4583,  4605,  4649,  4980,
         5031,  5400,  5493,  5494,  5495,  5875,  5887,  5888,  5895,
         5897,  5921,  5922,  6075,  6094,  6096,  6141,  6266,  6267,
         6268,  6451,  6490,  6491,  6492,  6700,  6779,  6986,  6987,
         7104,  7150,  7234,  7235,  7236,  7334,  7335,  7336,  7845,
         7846,  7912,  7970,  7971,  7972,  8119,  8120,  8121,  8405,
         8416,  8417,  8566,  8567,  8746,  9166,  9590,  9591,  9754,
         9755,  9941,  9942,  9966,  9971,  9972, 10032, 10197, 10635,
        10636, 10679, 10711, 10712, 10773, 10774, 10909, 10910, 11426,
      

We can also add new data to our table. Add the Diabetes data to new_table:

In [ ]:
new_table["DIABETES"] = fram.loc[SMOKER_BMI_rows,['DIABETES']]
new_table

,AGE,TOTCHOL,HEARTRTE,DIABETES
192,42,206.0,70.0,0
193,48,208.0,75.0,0
231,61,326.0,57.0,0
672,60,180.0,88.0,1
1102,44,229.0,104.0,0
...,...,...,...,...
10245,57,254.0,72.0,0
10246,64,199.0,95.0,1
10531,62,358.0,110.0,1
10941,54,246.0,92.0,0


## Simple Analyses of the Dataset
We will now answer a few questions about the Framingam data set.
1.) How many patients over 45 have a BMI over 40? 

In [ ]:
patients = fram.loc[((fram["AGE"]>45) & (fram.BMI>40)),["AGE","BMI"]]
patients.shape

(65, 2)

Another method:


In [ ]:
sum((fram.AGE>45) & (fram.BMI>40))


65

2.) How many data points are missing from the patient cholesterol? What is the median value for the cholesterol data that we do have?

In [ ]:
fram.index[fram["TOTCHOL"].isnull()].size

409

In [ ]:
fram["TOTCHOL"].median()

238.0

3.) Create a structure with all of the patient IDs and find the number of unique IDs.


In [ ]:
ID = fram["RANDID"]
num_ID = ID.size   # total number of IDs
num_ID

11627

In [ ]:
num_unq = ID.unique().size   # number of unique IDs
num_unq

4434

## Sorting Through Our Data
### Intersect
The intersect function can find the overlap between lists of numbers or strings. This is helpful for finding data that satisfies multiple criteria. 
For example, find the ID numbers of patients over the age of 40 and the ID numbers for patients with glucose levels over 130 mg/dL. Then use intersect to find the overlap in these lists.


In [ ]:
age_IDs = fram.loc[fram["AGE"] > 40,"RANDID"]
glucose_IDs = fram.loc[fram["GLUCOSE"] > 130, "RANDID"]
age_glucose_IDs = pd.merge(age_IDs, glucose_IDs, how="inner", on="RANDID")
age_glucose_IDs.drop_duplicates()

,RANDID
0,23727
3,43770
6,83398
12,95541
16,97026
...,...
711,9901360
714,9941222
716,9961615
719,9964282


Notice that intersect sorts the overlapping ID numbers. We can also specify to output the indices from each list where the matching values occur. (This can be implemented by `numpy` better)

In [ ]:
age_glucose_IDs, ia ,ib = np.intersect1d(age_IDs.to_numpy(), glucose_IDs.to_numpy(), return_indices=True)
age_glucose_IDs, ia ,ib

(array([  23727,   43770,   83398,   95541,   97026,  162207,  170881,
         205391,  210362,  276073,  359422,  411906,  414678,  524819,
         575116,  610021,  610146,  614850,  621362,  673362,  760664,
         771908,  821193,  860992,  891658,  937484,  968222, 1002586,
        1044236, 1155474, 1190905, 1257338, 1304801, 1346916, 1554708,
        1648625, 1692255, 1709566, 1735368, 1842886, 1875965, 1965250,
        2047933, 2055792, 2067160, 2068456, 2080190, 2157176, 2175833,
        2199791, 2353950, 2390122, 2409397, 2437351, 2439832, 2460331,
        2483517, 2577634, 2603578, 2656361, 2698297, 2708769, 2720671,
        2723598, 2924328, 2927695, 2976595, 3026761, 3065845, 3155718,
        3187190, 3236495, 3250443, 3366982, 3429166, 3429218, 3603542,
        3647049, 3735990, 3830539, 3906438, 3923558, 4072924, 4307282,
        4370066, 4452956, 4494395, 4511124, 4521105, 4659800, 4690988,
        4702754, 4726021, 4728926, 4730922, 4751304, 4757073, 4899712,
      

### RegExp
The regexp function is used to find patterns in a data set. It is often used with string arrays, but can be used with numeric data as well. There are many metacharacters and operators which can be used to specify what pattern you are looking for. For a full list of these options, use the help function.

Use regexp to find all of the ID numbers that begin with the number 1. 

First, we must convert the array of ID numbers to a string array:


In [ ]:
IDS = fram["RANDID"].to_string(index=False, header=False)
IDS

'    2448\n    2448\n    6238\n    6238\n    6238\n    9428\n    9428\n   10552\n   10552\n   11252\n   11252\n   11252\n   11263\n   11263\n   11263\n   12629\n   12629\n   12806\n   12806\n   12806\n   14367\n   14367\n   14367\n   16365\n   16365\n   16365\n   16799\n   16799\n   16799\n   19304\n   20375\n   20375\n   23727\n   23727\n   23727\n   24721\n   24721\n   24721\n   30928\n   33077\n   33077\n   33077\n   33555\n   34689\n   34689\n   34689\n   36459\n   36459\n   36459\n   40435\n   40435\n   40435\n   43522\n   43522\n   43522\n   43770\n   43770\n   43770\n   45464\n   45464\n   45464\n   47561\n   47561\n   47561\n   54224\n   55965\n   55965\n   55965\n   63156\n   63156\n   63156\n   63221\n   66472\n   66472\n   66472\n   67905\n   67905\n   67905\n   68194\n   68194\n   68194\n   68397\n   68397\n   68397\n   69134\n   70948\n   70948\n   70948\n   76273\n   77492\n   77492\n   77492\n   82188\n   82188\n   82188\n   82425\n   82425\n   82425\n   83398\n   83398\

Next, we specify the pattern to search for. We will use "\d*" to specify that any number of digits may follow the first number, 1:

In [ ]:
A = np.array(re.findall("1\d*", IDS))
A

array(['10552', '10552', '11252', ..., '12', '12', '12'], dtype='<U7')

Now find the IDs that begin with 1 and end with 5.


In [ ]:
A = np.array(re.findall("1\d*5", IDS))
A

array(['1055', '1055', '1125', ..., '15', '15', '15'], dtype='<U7')

Next find the IDs that begin with 1, end with 5, and have 23 at least one digit after the first number. To specify the new restraint, we can add "\d+23" to the pattern.


In [ ]:
A = np.array(re.findall("1\d+23\d*5", IDS))
A

array(['112305', '112305', '112305', '152375', '152375', '152375',
       '1723785', '1723785', '1723785'], dtype='<U7')

We will now try using re.match with alphabetic characters as opposed to numeric. 

Find all of the variable names in the Framingham data set which begin with the word "TIME".



In [ ]:
match_TIME = [re.match('^(TIME)\w*', column) is not None for column in fram.columns ]
match_TIME

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

The output match_TIME shows us which cells match the specificied pattern. In order to see the actual matching variable names, we can use the below for loop to remove the empty (non-matching) cells.


In [ ]:
fram.columns[match_TIME]

Index(['TIME', 'TIMEAP', 'TIMEMI', 'TIMEMIFC', 'TIMECHD', 'TIMESTRK',
       'TIMECVD', 'TIMEDTH', 'TIMEHYP'],
      dtype='object')

## Importing and Exporting Datasets
### Exporting
Pandas and Scipy have many functions for reading and writing all kind of data such as .txt, .dat, .csv, .xls

Write an Excel file which contains the data for all male smokers:

In [ ]:
male_smoker = fram[(fram.SEX==1) & (fram.CURSMOKE==1)]
male_smoker.to_excel('male_smoker.xls')

Now create an array of the male smoker table and write it to a csv file:


In [ ]:
male_smoker.to_csv('male_smoker.csv')

### Importing
Similar to exporting, the functions xlsread, dlmread, and csvread are no longer recommended in MATLAB 2019 for importing data. Instead, you should use readtable or readmatrix. Import the .xls and .csv files that we just created:

In [ ]:
pd.read_csv("male_smoker.csv")

,Unnamed: 0,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,educ,PREVCHD,PREVAP,PREVMI,PREVSTRK,PREVHYP,TIME,PERIOD,HDLC,LDLC,DEATH,ANGINA,HOSPMI,MI_FCHD,ANYCHD,STROKE,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
0,5,9428,1,245.0,48,127.5,80.0,1,20.0,25.34,0,0.0,75.0,70.0,1.0,0,0,0,0,0,0,1,NaN,NaN,0,0,0,0,0,0,0,0,8766,8766,8766,8766,8766,8766,8766,8766
1,6,9428,1,283.0,54,141.0,89.0,1,30.0,25.34,0,0.0,75.0,87.0,1.0,0,0,0,0,0,2199,2,NaN,NaN,0,0,0,0,0,0,0,0,8766,8766,8766,8766,8766,8766,8766,8766
2,23,16365,1,225.0,43,162.0,107.0,1,30.0,23.61,0,0.0,93.0,88.0,1.0,0,0,0,0,1,0,1,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,0
3,30,20375,1,294.0,46,142.0,94.0,1,15.0,26.31,0,0.0,98.0,64.0,1.0,0,0,0,0,1,0,1,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,0
4,31,20375,1,288.0,52,165.0,92.0,1,10.0,26.44,0,0.0,105.0,71.0,1.0,0,0,0,0,1,2277,2,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2589,11608,9984683,1,313.0,50,179.0,92.0,1,1.0,25.97,0,0.0,66.0,86.0,1.0,0,0,0,0,1,0,1,NaN,NaN,1,1,1,1,1,0,1,1,1884,1884,1884,1884,4300,1884,4300,0
2590,11609,9984683,1,NaN,56,157.5,89.0,1,1.0,24.85,0,0.0,85.0,NaN,1.0,1,1,1,0,1,2093,2,NaN,NaN,1,1,1,1,1,0,1,1,1884,1884,1884,1884,4300,1884,4300,0
2591,11610,9989287,1,207.0,51,126.5,80.0,1,43.0,19.71,0,0.0,65.0,68.0,3.0,0,0,0,0,0,0,1,NaN,NaN,1,0,0,0,0,0,0,0,7746,7746,7746,7746,7746,7746,7746,7746
2592,11611,9989287,1,210.0,57,116.0,69.0,1,30.0,17.65,0,0.0,70.0,68.0,3.0,0,0,0,0,0,2148,2,NaN,NaN,1,0,0,0,0,0,0,0,7746,7746,7746,7746,7746,7746,7746,7746


In [ ]:
pd.read_excel("male_smoker.xls")

,Unnamed: 0,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,educ,PREVCHD,PREVAP,PREVMI,PREVSTRK,PREVHYP,TIME,PERIOD,HDLC,LDLC,DEATH,ANGINA,HOSPMI,MI_FCHD,ANYCHD,STROKE,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
0,5,9428,1,245.0,48,127.5,80.0,1,20.0,25.34,0,0.0,75.0,70.0,1.0,0,0,0,0,0,0,1,NaN,NaN,0,0,0,0,0,0,0,0,8766,8766,8766,8766,8766,8766,8766,8766
1,6,9428,1,283.0,54,141.0,89.0,1,30.0,25.34,0,0.0,75.0,87.0,1.0,0,0,0,0,0,2199,2,NaN,NaN,0,0,0,0,0,0,0,0,8766,8766,8766,8766,8766,8766,8766,8766
2,23,16365,1,225.0,43,162.0,107.0,1,30.0,23.61,0,0.0,93.0,88.0,1.0,0,0,0,0,1,0,1,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,0
3,30,20375,1,294.0,46,142.0,94.0,1,15.0,26.31,0,0.0,98.0,64.0,1.0,0,0,0,0,1,0,1,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,0
4,31,20375,1,288.0,52,165.0,92.0,1,10.0,26.44,0,0.0,105.0,71.0,1.0,0,0,0,0,1,2277,2,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2589,11608,9984683,1,313.0,50,179.0,92.0,1,1.0,25.97,0,0.0,66.0,86.0,1.0,0,0,0,0,1,0,1,NaN,NaN,1,1,1,1,1,0,1,1,1884,1884,1884,1884,4300,1884,4300,0
2590,11609,9984683,1,NaN,56,157.5,89.0,1,1.0,24.85,0,0.0,85.0,NaN,1.0,1,1,1,0,1,2093,2,NaN,NaN,1,1,1,1,1,0,1,1,1884,1884,1884,1884,4300,1884,4300,0
2591,11610,9989287,1,207.0,51,126.5,80.0,1,43.0,19.71,0,0.0,65.0,68.0,3.0,0,0,0,0,0,0,1,NaN,NaN,1,0,0,0,0,0,0,0,7746,7746,7746,7746,7746,7746,7746,7746
2592,11611,9989287,1,210.0,57,116.0,69.0,1,30.0,17.65,0,0.0,70.0,68.0,3.0,0,0,0,0,0,2148,2,NaN,NaN,1,0,0,0,0,0,0,0,7746,7746,7746,7746,7746,7746,7746,7746


## Handling Big Datasets
### Head and Tail

The head and tail functions can be used to view the first and last rows of a table or array. The default for heads and tails is to display the first and last eight rows, respectively. However, the number of rows can be altered for each function. Consider the framingham dataset we used earlier:


In [ ]:
fram.head(5)

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,educ,PREVCHD,PREVAP,PREVMI,PREVSTRK,PREVHYP,TIME,PERIOD,HDLC,LDLC,DEATH,ANGINA,HOSPMI,MI_FCHD,ANYCHD,STROKE,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
0,2448,1,195.0,39,106.0,70.0,0,0.0,26.97,0,0.0,80.0,77.0,4.0,0,0,0,0,0,0,1,NaN,NaN,0,0,1,1,1,0,1,0,8766,6438,6438,6438,8766,6438,8766,8766
1,2448,1,209.0,52,121.0,66.0,0,0.0,NaN,0,0.0,69.0,92.0,4.0,0,0,0,0,0,4628,3,31.0,178.0,0,0,1,1,1,0,1,0,8766,6438,6438,6438,8766,6438,8766,8766
2,6238,2,250.0,46,121.0,81.0,0,0.0,28.73,0,0.0,95.0,76.0,2.0,0,0,0,0,0,0,1,NaN,NaN,0,0,0,0,0,0,0,0,8766,8766,8766,8766,8766,8766,8766,8766
3,6238,2,260.0,52,105.0,69.5,0,0.0,29.43,0,0.0,80.0,86.0,2.0,0,0,0,0,0,2156,2,NaN,NaN,0,0,0,0,0,0,0,0,8766,8766,8766,8766,8766,8766,8766,8766
4,6238,2,237.0,58,108.0,66.0,0,0.0,28.50,0,0.0,80.0,71.0,2.0,0,0,0,0,0,4344,3,54.0,141.0,0,0,0,0,0,0,0,0,8766,8766,8766,8766,8766,8766,8766,8766


In [ ]:
fram.tail(3)

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,educ,PREVCHD,PREVAP,PREVMI,PREVSTRK,PREVHYP,TIME,PERIOD,HDLC,LDLC,DEATH,ANGINA,HOSPMI,MI_FCHD,ANYCHD,STROKE,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
11624,9999312,2,196.0,39,133.0,86.0,1,30.0,20.91,0,0.0,85.0,80.0,3.0,0,0,0,0,0,0,1,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,4201
11625,9999312,2,240.0,46,138.0,79.0,1,20.0,26.39,0,0.0,90.0,83.0,3.0,0,0,0,0,0,2390,2,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,4201
11626,9999312,2,NaN,50,147.0,96.0,1,10.0,24.19,0,0.0,94.0,NaN,3.0,0,0,0,0,1,4201,3,NaN,NaN,0,0,0,0,0,0,0,1,8766,8766,8766,8766,8766,8766,8766,4201


### Sparce Matrices

Sparse matrices are useful when your dataset is comprised of mostly zeros. As opposed to normal matrices which store every element in the matrix, sparce matrices only store the nonzero elements, along with their row indices. Sparce matrices therefor require much less memory for storage than full matrices.

The first step in creating a sparse matrix is determining the density of nonzero elements. The lower the density, the more it makes sense to create a sparce matrix. Consider the variables in the Framingham dataset which detail prevalent diseases:
- Prevalent Angina Pectoris (PREVAP)
- Prevalent Coronary Heart Disease (PREVCHD)
- Prevalent Myocardial Infarction (PREVMI)
- Prevalent Stroke (PREVSTRK)
- Prevalent Hypertensive (PREVHYP)

The section of the data set containing these variables is mostly comprised of zeros. Create an array that includes these five variables, find the density, and find the memory required for the table.


In [ ]:
prev_table = fram[['PREVAP','PREVCHD','PREVHYP','PREVMI','PREVSTRK']]
prev_array = prev_table.values

For calculating the density, we can use np.count_nonzero to find the number of nonzero elements and np.size to find the total number of elements in the matrix. Then use who to display the required memory.

In [ ]:
np.count_nonzero(prev_array) / np.size(prev_array)

0.1262406467704481

In [ ]:
prev_array.nbytes

465080

The memory required for the prev_array variable is 465 kB. Now, convert prev_array into a sparse matrix. You will see that the output is an array of the nonzero elements and their respective indeces, sorted by column. What happens to the requried storage space of the array?


In [ ]:
S = sparse.csr_matrix(prev_array)
S.data.nbytes

58712

The size of the array decreased to 58.7 kB, or about 12% of the original required memory. 

If necessary, you can then convert the sparse matrix back into the full matrix with the full command:

In [ ]:
A = S.toarray()
A.nbytes

465080

It is also possible to create a sparse matrix directly from the nonzero elements, without needing the full matrix:

csr_matrix((data, (row_ind, col_ind)), \[shape=(M, N)\])

where data, row_ind and col_ind satisfy the relationship a\[row_ind\[k\], col_ind\[k\]\] = data\[k\].

In [ ]:
data = np.array([1,2,3,4,5])
row = np.array([2, 1, 2, 3, 0])
col = np.array([0, 1, 1, 2, 3])
sp = sparse.csr_matrix((data, (row, col)), shape=(4, 4))
sp.toarray()

array([[0, 0, 0, 5],
       [0, 2, 0, 0],
       [1, 3, 0, 0],
       [0, 0, 4, 0]], dtype=int64)